# Check existing BBP simulation campaigns on AWS

In [13]:
import json
import subprocess
import os

In [28]:
# AWS paths
aws_s3 = "s3://openbluebrain"
aws_root_staging = os.path.join(aws_s3, "staging/Simulation_data/Simulation_Campaigns/Rat/SSCx")
aws_root = os.path.join(aws_s3, "Simulation_data/Simulation_Campaigns/Rat/SSCx")

In [4]:
# Interesting campaigns
list_of_campaigns = [
    {"name": "assemblies", "gpfs_path": "/gpfs/bbp.cscs.ch/project/proj96/home/ecker/simulations/d21efd45-7740-4268-b06f-e1de35f2b6cf", "aws_hash": "0fab8d2c2d", "circuit": "nbS1-O1"},
    {"name": "SM-recalib", "gpfs_path": "/gpfs/bbp.cscs.ch/project/proj83/home/isbister/simulations/elife_sscx_physiology_2024/9-SchneiderMizell/0_recalibration/0_0_recalibration_6thIter_20_3_24", "aws_hash": "09151af2d2", "circuit": "nbS1-O1-SM"},
    {"name": "SM-whisker-scan", "gpfs_path": "/gpfs/bbp.cscs.ch/project/proj83/home/isbister/simulations/elife_sscx_physiology_2024/9-SchneiderMizell/1_whisker_deflections/1_3_whisker_deflections_scan_16_4_24", "aws_hash": "3f6517825f", "circuit": "nbS1-O1-SM"},
    {"name": "SM-whisker-soma", "gpfs_path": "/gpfs/bbp.cscs.ch/project/proj83/home/isbister/simulations/elife_sscx_physiology_2024/9-SchneiderMizell/2_whisker_deflections_soma/2_whisker_deflections_soma_0__19_11_24", "aws_hash": "f132b97aa0", "circuit": "nbS1-O1-SM"},
    {"name": "spikesorting", "gpfs_path": "/gpfs/bbp.cscs.ch/project/proj83/home/isbister/simulations/spike_sorting/8_spikesorting_stimulus_test_neuropixels_13-4-24/8_spikesorting_stimulus_test_neuropixels_13-4-24__8slc_100f_72r_250t_200ms_0", "aws_hash": "359d0ebe7d", "circuit": "nbS1-O1"},
]

# Extra campaigns
# list_of_campaigns = [
#     {"name": "beta-spont", "gpfs_path": "/gpfs/bbp.cscs.ch/project/proj83/home/pokorny/SimplifiedConnectomeModels/simulations_v2/SSCx-HexO1-Release-TC__Spont", "aws_hash": "eb7f4d64c3", "circuit": "nbS1-O1-beta"},
#     {"name": "beta-evoked", "gpfs_path": "/gpfs/bbp.cscs.ch/project/proj83/home/pokorny/SimplifiedConnectomeModels/simulations_v2/SSCx-HexO1-Release-TC__Evoked", "aws_hash": "35a7da37a3", "circuit": "nbS1-O1-beta"},
# ]


In [9]:
for cmpgn in list_of_campaigns:
    print(f"CHECKING '{cmpgn['name']}'...")
    try:
        aws_path = os.path.join(aws_root, cmpgn['aws_hash'])
        output = subprocess.check_output(f"aws s3 ls {aws_path} --no-sign-request --recursive --human-readable", shell=True, text=True)
    except:
        output = None  # Not found on AWS

    if output is None:
        try:
            aws_path = os.path.join(aws_root_staging, cmpgn['aws_hash'])
            output = subprocess.check_output(f"aws s3 ls {aws_path} --no-sign-request --recursive --human-readable", shell=True, text=True)
        except:
            output = None  # Not found on AWS staging

    if output is None:
        print(f"ERROR: '{cmpgn['name']}' not found on AWS!")
        cmpgn["aws_path"] = None
        cmpgn["aws_content"] = None
        cmpgn["format"] = None
    else:
        cmpgn["aws_path"] = aws_path
        cmpgn["aws_content"] = output
        if "BlueConfig" in output and not "simulation_config.json" in output:
            cmpgn["format"] = "legacy"
        elif "BlueConfig" not in output and "simulation_config.json" in output:
            cmpgn["format"] = "SONATA"
        else:
            cmpgn["format"] = "unknown"


CHECKING 'assemblies'...
CHECKING 'SM-recalib'...
CHECKING 'SM-whisker-scan'...
CHECKING 'SM-whisker-soma'...
CHECKING 'spikesorting'...


In [61]:
# Get individual files & number of simulations
def get_files(content):
    files = content.split("\n")
    rel_path = os.path.relpath(cmpgn["aws_path"], aws_s3)
    files = [file[file.find(rel_path):] for file in files if len(file) > 0]
    files = [file.replace(rel_path, cmpgn["aws_path"]) for file in files]
    return files

def get_num_simulations(files, legacy=False):
    if legacy:
        cfg_fn = "BlueConfig"
        out_fn = "out.h5"
    else:
        cfg_fn = "simulation_config.json"
        out_fn = "spikes.h5"
    cfg_files = [file for file in files if cfg_fn in file]
    out_files = [file for file in files if out_fn in file]
    if len(out_files) > len(cfg_files):
        return len(cfg_files)
    else:
        return len(out_files)

for cmpgn in list_of_campaigns:
    cmpgn["files"] = get_files(cmpgn["aws_content"])
    cmpgn["num_files"] = len(cmpgn["files"])
    cmpgn["num_simulations"] = get_num_simulations(cmpgn["files"], legacy=cmpgn["format"] == "legacy")

In [62]:
# Write to .json
with open("AWS_campaigns.json", "x") as f:
    json.dump(list_of_campaigns, f)

In [22]:
# Print folder content
for cmpgn in list_of_campaigns:
    print(f"{cmpgn['name']} ({cmpgn['format']}): {cmpgn['aws_path']}")
    print(f"{cmpgn['aws_content']}".strip())
    print()

assemblies (legacy): s3://openbluebrain/staging/Simulation_data/Simulation_Campaigns/Rat/SSCx/0fab8d2c2d
2024-11-27 06:31:35    5.4 KiB staging/Simulation_data/Simulation_Campaigns/Rat/SSCx/0fab8d2c2d/0/BlueConfig
2024-11-27 06:31:35    5.4 KiB staging/Simulation_data/Simulation_Campaigns/Rat/SSCx/0fab8d2c2d/1/BlueConfig
2024-11-27 06:31:36    5.4 KiB staging/Simulation_data/Simulation_Campaigns/Rat/SSCx/0fab8d2c2d/2/BlueConfig
2024-11-27 06:31:37    5.4 KiB staging/Simulation_data/Simulation_Campaigns/Rat/SSCx/0fab8d2c2d/3/BlueConfig
2024-11-27 06:31:38    5.4 KiB staging/Simulation_data/Simulation_Campaigns/Rat/SSCx/0fab8d2c2d/4/BlueConfig
2024-11-27 06:31:39    5.4 KiB staging/Simulation_data/Simulation_Campaigns/Rat/SSCx/0fab8d2c2d/5/BlueConfig
2024-11-27 06:31:41    5.4 KiB staging/Simulation_data/Simulation_Campaigns/Rat/SSCx/0fab8d2c2d/6/BlueConfig
2024-11-27 06:31:41    5.4 KiB staging/Simulation_data/Simulation_Campaigns/Rat/SSCx/0fab8d2c2d/7/BlueConfig
2024-11-27 06:31:41    

In [ ]:
# # Download config files
# output_root = "/Users/pokorny/Downloads/AWS_campaigns"
# if not os.path.exists(output_root):
#     os.makedirs(output_root)

# for cmpgn in list_of_campaigns:
#     try:
#         aws_path = cmpgn["aws_path"]
#         # aws s3 cp s3://openbluebrain/file.xyz /local/path/file.xyz --no-sign-request 
#         # output = subprocess.check_output(f"aws s3 cp {aws_path} {output_root} --no-sign-request --recursive ", shell=True, text=True)
#     except:
#         output = None  # Not found on AWS staging
